<center><h2>ALTEGRAD Project</h2>

<hr>
<span style="font-variant: small-caps;">Xavier Jiménez, Jean Quentin, Sacha Revol</span><br>
<hr>
</center>

# Imports

In [7]:
import networkx as nx
import csv
import numpy as np
from random import randint
from sklearn.linear_model import LogisticRegression
import torch

from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd

from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader
from transformers import AutoModel, AutoTokenizer


# Load data

In [10]:
node_pairs = pd.read_csv('data/training_data_new/train_val_node_pairs.csv', header=None)
labels = pd.read_csv('data/training_data_new/train_val_labels.csv', header=None)
        
train_node_pairs, val_node_pairs, train_labels, val_labels = train_test_split(node_pairs, 
                                                                              labels, 
                                                                              train_size=0.8, 
                                                                              test_size=0.2, 
                                                                              random_state=42)

train_node_pairs = train_node_pairs.reset_index(drop=True)
val_node_pairs = val_node_pairs.reset_index(drop=True)
train_labels = train_labels.reset_index(drop=True)
val_labels = val_labels.reset_index(drop=True)


In [11]:
test_node_pairs = pd.read_csv('data/training_data_new/train_test_node_pairs.csv', header=None)
test_labels = pd.read_csv('data/training_data_new/train_test_labels.csv', header=None)


In [12]:
len(train_node_pairs)#+len(val_node_pairs)+len(test_node_pairs))

279539

In [13]:
# Read the abstract of each paper
abstracts = dict()
with open('data/abstracts.txt', 'r') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract


In [30]:
length_total = train_node_pairs.shape[0]


## Finetuning embeddings for sentence transformers

In [52]:


abstracts_all = []

length_total = train_node_pairs.shape[0]

for i in tqdm(range(length_total)):
    #print(i)
    abstract_1 = int(train_node_pairs.iloc[[i]][0].item())
    abstract_2 = int(train_node_pairs.iloc[[i]][1].item())
    
    linked = train_labels.iloc[[i]][0].item()

    if linked==1:
        abstracts_all.append(InputExample(texts=[abstracts[abstract_1], abstracts[abstract_2]], label=1.0))
    else:
        abstracts_all.append(InputExample(texts=[abstracts[abstract_1], abstracts[abstract_2]], label=0.0))


100%|██████████| 279539/279539 [01:45<00:00, 2647.75it/s]


In [53]:
torch.cuda.empty_cache()


#Define your train dataset, the dataloader and the train loss
train_loss = losses.CosineSimilarityLoss(model)

length_total_training = len(abstracts_all)

for i in range(10):
    print("Part: " + str(i))
    #Tune the model
    sub = abstracts_all[i*length_total_training//10:(i+1)*length_total_training//10]
    print("sublength : " + str(len(sub)))
    
    train_dataloader = DataLoader(sub, shuffle=True, batch_size=5)
    model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)
    torch.save(model.state_dict(), "model_pieces/specter_finetuned_full_"+str(i))


Part: 0
sublength : 27953


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5591 [00:00<?, ?it/s]

Part: 1
sublength : 27954


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

...

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5591 [00:00<?, ?it/s]

Part: 9
sublength : 27954


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5591 [00:00<?, ?it/s]

In [55]:
torch.save(model.state_dict(), "models/specter_finetuned_full")


In [14]:
model = SentenceTransformer('sentence-transformers/allenai-specter')
model.load_state_dict(torch.load("models/specter_finetuned_full"))
model.eval()


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

# Reading data

In [24]:
# Create a graph
G = nx.read_edgelist('data/edgelist.txt', delimiter=',', create_using=nx.Graph(), nodetype=int)
nodes = list(G.nodes())
n = G.number_of_nodes()
m = G.number_of_edges()
print('Number of nodes:', n)
print('Number of edges:', m)


Exception ignored in: <function ZipFile.__del__ at 0x7f408200fd30>
Traceback (most recent call last):
  File "/opt/conda/envs/altegrad2/lib/python3.8/zipfile.py", line 1821, in __del__
    self.close()
  File "/opt/conda/envs/altegrad2/lib/python3.8/zipfile.py", line 1838, in close
    self.fp.seek(self.start_dir)
ValueError: seek of closed file


Number of nodes: 138499
Number of edges: 1091955


In [25]:
# Read the abstract of each paper
abstracts = dict()
with open('data/abstracts.txt', 'r') as f:
    for line in f:
        node, abstract = line.split('|--|')
        abstracts[int(node)] = abstract


# Computing embeddings

In [ ]:

model_name = "scibert_sentence_transformers"

txt2feat = dict()
nodes = list(abstracts.keys())

article_textual_embeddings = []


if model_name == "bert":
    
    tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
    model = AutoModel.from_pretrained('bert-base-uncased').to("cuda")

    model.cuda()
    model.eval()
    
    for i in tqdm(nodes):
        index = int(i)

        tokens = tokenizer.encode(abstracts[index], return_tensors='pt', truncation=True, max_length=250).to("cuda")
        node_paper_embedding_1 = model(tokens).pooler_output.detach().cpu().numpy()

        txt2feat[index] = node_paper_embedding_1

        
elif model_name == "scibert":
    
    tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
    model = AutoModel.from_pretrained('allenai/scibert_scivocab_uncased').to("cuda")
    
    model.cuda()
    model.eval()
    
    for i in tqdm(nodes):
        index = int(i)

        tokens = tokenizer.encode(abstracts[index], return_tensors='pt', truncation=True, max_length=250).to("cuda")
        node_paper_embedding_1 = model(tokens).pooler_output.detach().cpu().numpy()

        txt2feat[index] = node_paper_embedding_1


elif model_name == "scibert_sentence_transformers":
    
    model = SentenceTransformer('gsarti/scibert-nli')

    model.cuda()
    model.eval()

    for i in tqdm(nodes):
        index = int(i)

        article_textual_embedding = model.encode(abstracts[index])

        txt2feat[index] = article_textual_embedding


elif model_name == "specter":

    model = SentenceTransformer('sentence-transformers/allenai-specter')

    model.cuda()
    model.eval()

    for i in tqdm(nodes):
        index = int(i)

        article_textual_embedding = model.encode(abstracts[index])

        txt2feat[index] = article_textual_embedding


for i in tqdm(nodes):
    index = int(i)
    article_textual_embeddings.append(txt2feat[index])


Saving options

In [86]:
torch.save(article_textual_embeddings, f'data/article_textual_embeddings.pt')

In [ ]:
article_textual_embeddings = torch.load(f'/data/article_textual_embeddings.pt')

In [7]:
npz_to_save = np.array(article_textual_embeddings3)

In [70]:
with open(f'data/embeddings_19_02_best_1.npy', 'wb') as f:
    np.save(f, npz_to_save)

In [3]:
with open(f'data/embeddings_19_02.npy', 'rb') as f:
    article_textual_embeddings3 = np.load(f)

Splitting for easier download

In [5]:
article_textual_embeddings3_part1 = article_textual_embeddings3[:138499//2]
npz_to_save_1 = np.array(article_textual_embeddings3_part1)

article_textual_embeddings3_part2 = article_textual_embeddings3[138499//2:]
npz_to_save_2 = np.array(article_textual_embeddings3_part2)

In [6]:
with open(f'data/embeddings_19_02_best_part_1.npy', 'wb') as f:
    np.savez_compressed(f, npz_to_save_1)

with open(f'data/embeddings_19_02_best_part_2.npy', 'wb') as f:
    np.savez_compressed(f, npz_to_save_2)

Checking that the split worked

In [9]:
with open(f'data/embeddings_19_02_best_1.npy', 'wb') as f:
    np.savez_compressed(f, npz_to_save)
    #np.save(f, npz_to_save)

In [7]:
with open(f'data/embeddings_19_02_best_part_1.npy', 'rb') as f:
    article_textual_embeddings3_part1 = np.load(f)
    part1 = article_textual_embeddings3_part1['arr_0']

with open(f'data/embeddings_19_02_best_part_2.npy', 'rb') as f:
    article_textual_embeddings3_part2 = np.load(f)
    part2 = article_textual_embeddings3_part2['arr_0']

full_array = np.concatenate((part1,part2), axis=0)

## Evaluating performance

Computing positive and negative samples

In [ ]:
pos_samples = []
neg_samples = []
length_total = val_node_pairs.shape[0]

for i in tqdm(range(length_total)):

    linked = val_labels.iloc[[i]][0].item()

    if linked==1:
        pos_samples.append([int(val_node_pairs.iloc[[i]][0].item()),int(val_node_pairs.iloc[[i]][1].item())])
    else:
        neg_samples.append([int(val_node_pairs.iloc[[i]][0].item()),int(val_node_pairs.iloc[[i]][1].item())])


Performance on negatives

In [45]:
from sklearn.metrics.pairwise import cosine_similarity


sims = []
for neg_sample in tqdm(neg_samples[:500]):
    
    ###if embeddings are precomputed run this
    #node_paper_embedding_1 = np.expand_dims(full_array[neg_sample[0]], axis=0)
    #node_paper_embedding_2 = np.expand_dims(full_array[neg_sample[1]], axis=0)

    ###otherwise we compute them on the fly
    if model_name == "bert" or model_name == "scibert":
        tokens = tokenizer.encode(abstracts[neg_sample[0]], return_tensors='pt', truncation=True, max_length=250).to("cuda")
        node_paper_embedding_1 = model(tokens).pooler_output.detach().cpu().numpy()

        tokens = tokenizer.encode(abstracts[neg_sample[1]], return_tensors='pt', truncation=True, max_length=250).to("cuda")
        node_paper_embedding_2 = model(tokens).pooler_output.detach().cpu().numpy()

    else:
        node_paper_embedding_1 = np.expand_dims(model.encode(abstracts[neg_sample[0]]), axis=0)
        node_paper_embedding_2 = np.expand_dims(model.encode(abstracts[neg_sample[1]]), axis=0)

    
    paper_sim = cosine_similarity(node_paper_embedding_1, node_paper_embedding_2)[0]
    #print(paper_sim)
    sims.append(paper_sim)


print(sum(sims)/len(sims))
    

100%|██████████| 500/500 [00:17<00:00, 27.88it/s]

[0.5569189]


Performance on positives

In [47]:
from sklearn.metrics.pairwise import cosine_similarity


sims = []
for pos_sample in tqdm(pos_samples[:500]):
    
    ###if embeddings are precomputed run this
    #node_paper_embedding_1 = np.expand_dims(full_array[pos_sample[0]], axis=0)
    #node_paper_embedding_2 = np.expand_dims(full_array[pos_sample[1]], axis=0)

    if model_name == "bert" or model_name == "scibert":
        tokens = tokenizer.encode(abstracts[pos_sample[0]], return_tensors='pt', truncation=True, max_length=250).to("cuda")
        node_paper_embedding_1 = model(tokens).pooler_output.detach().cpu().numpy()

        tokens = tokenizer.encode(abstracts[pos_sample[1]], return_tensors='pt', truncation=True, max_length=250).to("cuda")
        node_paper_embedding_2 = model(tokens).pooler_output.detach().cpu().numpy()

    else:
        node_paper_embedding_1 = np.expand_dims(model.encode(abstracts[pos_sample[0]]), axis=0)
        node_paper_embedding_2 = np.expand_dims(model.encode(abstracts[pos_sample[1]]), axis=0)

    
    sims.append(cosine_similarity(node_paper_embedding_1, node_paper_embedding_2)[0])

print(sum(sims)/len(sims))
    

100%|██████████| 500/500 [00:19<00:00, 25.75it/s]

[0.70524037]
